This is my first version of EC-DG-CA3 network. Hector Sainvet, April 2022.

Local (not large-scale model)

In [1]:
# libraries
import numpy as np
import numpy.matlib
import pandas 
import brian2
import watermark

import os

In [2]:
#Graphic
import matplotlib.pyplot as plt
from matplotlib import cm
width_in_inches = 12
height_in_inches = 8
dots_per_inch = 70

# Parameters

In [3]:
parameters = {}

#Number of cells
parameters['n_mEC'] = 50000
parameters['n_DG'] = 500000
parameters['n_CA3'] = 200000
parameters['n_IN'] = 2500

#Connectivity
parameters['c_EI']= 0.1
parameters['c_IE']= 0.3
parameters['sigma_EI']= 150 * 10^(-6) 
parameters['sigma_IE'] = 300 * 10^(-6) 
parameters['J_EI'] = 8 * 10^(-9) 
parameters['J_IE'] = 0.0025 #normalized ?

### Time

In [4]:
parameters.update({ 'dt' : 0.5 * brian2.ms,  # timestep 
                    'trial_length' : 6000 * brian2.ms, # trial length (s),
                   'start_time': 0 *brian2.second, # s
                   'end_time': 10 *brian2.second}) 

# Physiology

Define the function that transforms input current to changes in firing rate for excitatory neurons (Abbott and Chance, 2005). 
<br>
$$r_E = \frac{aI_{syn,E} - b}{1 - e^{-d(aI_{syn,E} - b)}} $$

Update the firing rates of the interneurons using a threshold linear input/output function
$$ \begin{cases}
  r_I = \frac{1}{g_2}(c_II_{syn,I} - c_0) + r_0 & \text{for } r_I\ge 0\\    
  r_I = 0     & \text{otherwise }  
\end{cases} $$

In [5]:
def current_to_frequency(input_current,population_type,parameters):
    if population_type == 'E':
        a = parameters['a_E']
        b = parameters['b_E']
        d = parameters['d_E']
        return np.divide((a*input_current - b),(1 - np.exp(-d*(a*input_current - b))))
    if population_type == 'I':
        c_I = parameters['c_I']
        g_2 = parameters['g_2']
        r_0 = parameters['r0_I']
        c_0 = parameters['c_0']
        r = np.maximum((1/g_2)*(c_I*input_current - c_0) + r_0,0)
        return r
        # NOTE: using python broadcasting for ones subtraction, so should work for multi-area case too

# Connectivity

In [6]:
area_list = ['mEC','DG','CA3','PV']
area_column_list  = ['from '+ mystring for mystring in area_list]
area_row_list  = ['to '+ mystring for mystring in area_list]

Parameters from Guzman et al., 2021.

In [7]:
# Synaptic strengths from E cells
g_mEC_DG    = 200 * brian2.pA      
g_DG_CA3   = 50 * brian2.pA
g_DG_IN   = 50 * brian2.pA


# Synaptic strengths from PV cells
g_IN_DG    = - 75 * brian2.pA                                           

# Synaptic strengths from SST cells
g_PV_SST     = -100 * brian2.pA  


# put values into connectivity matrix
J = np.array([[0,            g_mEC_DG, 0,0],
              [0,    0 , g_DG_CA3 ,g_DG_IN],
               [0,   0  , 0,0],
              [0,   g_IN_DG        , 0,0]]) * brian2.amp

J = J.transpose()

J_NMDA = J*((J>0).astype(np.int))
J_GABA = J*((J<0).astype(np.int))

row_labels = ['to mEC', 'to DG','to CA3', 'to IN']
column_labels = ['from mEC', 'from DG','from CA3', 'to IN']

df_all = pandas.DataFrame(J, columns=column_labels, index=row_labels)

df_all

,from mEC,from DG,from CA3,to IN
to mEC,0.000000e+00,0.000000e+00,0.0,0.000000e+00
to DG,2.000000e-10,0.000000e+00,0.0,-7.500000e-11
to CA3,0.000000e+00,5.000000e-11,0.0,0.000000e+00
to IN,0.000000e+00,5.000000e-11,0.0,0.000000e+00


# Input

In [8]:
# Random

In [9]:
def generate_random_inputs(N):
    return np.random.choice([-1,1], size = (N,))

In [10]:
# Ecological

In [11]:
?

# Model

In [12]:
class Model():
    def __init__(self,parameter):
        self.parameters = parameter.copy()
    
    def run(self):
        self.__init__(self.parameters)
        
        #Time
        self.dt= self.parameters['dt']  
        self.num_iterations = int(self.parameters['trial_length']/self.dt)
        return
    
    def plot(self):
        return

# Trials

In [13]:
os.chdir(r"C:\Users\Hector\Documents\PhD\Stage_Fig")

In [26]:
save_name = 'bc_selected_distraction_0.pdf'


p_0 = parameters.copy()

model_0 = Model(p_0)
model_0.run()
fig = model_0.draw_selected()
fig.savefig(save_name)

KeyError: 'dt'